In [123]:
import numpy as np 
import pandas as pd

In [124]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [125]:
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [126]:
df=pd.read_csv("Hindi_English_Truncated_Corpus.csv")

In [127]:
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [128]:
df['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [129]:
df=df[df['source']=='ted']
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [130]:
pd.isnull(df).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [131]:
df=df[~pd.isnull(df['english_sentence'])]

In [132]:
df.drop_duplicates(inplace=True)

In [133]:
df=df.sample(n=25000,random_state=42)
df.shape

(25000, 3)

In [134]:
df['english_sentence']=df['english_sentence'].apply(lambda x: x.lower())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.lower())

In [135]:
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub("'", '', x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [136]:
exclude_punct=set(string.punctuation)
df['english_sentence']=df['english_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in exclude_punct))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in exclude_punct))

In [137]:
remove_digits = str.maketrans('', '', digits)
df['english_sentence']=df['english_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [138]:
df['hindi_sentence'] = df['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [139]:
df.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...


In [140]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [141]:
len(all_eng_words)


14030

In [142]:
len(all_hindi_words)

17540

In [143]:
df['length_eng_sentence']=df['english_sentence'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [144]:
df.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,START_ हम अभी तक नहीं जानते हैं कि उसके मातापि...,11,16
85038,ted,no keyboard,START_ कोई कुंजीपटल नहीं _END,2,5
58018,ted,but as far as being a performer,START_ लेकिन एक कलाकार होने के साथ _END,7,8
74470,ted,and this particular balloon,START_ और यह खास गुब्बारा _END,4,6
122330,ted,and its not as hard as you think integrate cli...,START_ और जितना आपको लगता है यह उतना कठिन नहीं...,16,20


In [145]:
df[df['length_eng_sentence']>30].shape

(0, 5)

In [146]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_hin_sentence']<=20]

In [147]:
df.shape

(24774, 5)

In [148]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [149]:
max_length_src=max(df['length_eng_sentence'])
max_length_tar=max(df['length_hin_sentence'])

In [150]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(14030, 17540)

In [151]:
num_decoder_tokens = num_decoder_tokens + 1 #for zero padding
num_encoder_tokens = num_encoder_tokens + 1

In [152]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [153]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [154]:
df=shuffle(df)

In [155]:
df.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
55399,ted,now i had an amazing experience i was years old,START_ अब मुझे एक अदभुत अनुभव हुआ मैं साल का थ...,10,12
92180,ted,the whole country went mad,START_ पूरा देश ही उथलपुथल में था। _END,5,8
45268,ted,in fact if the world were just filled with com...,START_ वास्तव में यदि संसार मात्र करुणा से भरा...,10,11
16544,ted,thanking us for the valiant effort displayed,START_ प्रदर्शन के लिए आभार प्रगट करने के लिए ...,7,12
52090,ted,that is all about execution all about the howto,START_ जो कि गरजने के बजाय बरसने का अध्याय है ...,9,19
77007,ted,then you get to their political articles,START_ अब इनके राजनैतिक लेख देखिये। _END,7,7
109851,ted,they didnt care about it,START_ उन्हें इश बात की पर्व नहीं थी _END,5,9
80472,ted,as simple as possible but no simpler,START_ को जितना हो सके आसान बनाये लेकिन साधारण...,7,11
30257,ted,and so amitabha decided that he would take tha...,START_ और इसलिये अमिताभ ने फ़ैसला किया कि वो इ...,10,13
62747,ted,a wellwisher of mine,START_ मेरा भला चाहने वाली _END,4,6


In [156]:
X, y = df['english_sentence'], df['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape,y_train.shape, y_test.shape


((19819,), (4955,), (19819,), (4955,))

In [157]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [158]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [159]:
latent_dim=300

In [160]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [161]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [162]:
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, None, 300)            4209300   ['input_7[0][0]']             
                                                                                                  
 embedding_7 (Embedding)     (None, None, 300)            5262300   ['input_8[0][0]']             
                                                                                            

In [163]:
from keras.optimizers import Adam
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [164]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, None, 300)            4209300   ['input_7[0][0]']             
                                                                                                  
 embedding_7 (Embedding)     (None, None, 300)            5262300   ['input_8[0][0]']             
                                                                                            

In [165]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100


In [166]:
model.fit(
    generate_batch(X_train, y_train, batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs
)

Epoch 1/100
154/154 [==============================] - 175s 1s/step - loss: 6.8068
Epoch 2/100
154/154 [==============================] - 164s 1s/step - loss: 6.0136
Epoch 3/100
154/154 [==============================] - 162s 1s/step - loss: 5.6594
Epoch 4/100
154/154 [==============================] - 164s 1s/step - loss: 5.4206
Epoch 5/100
154/154 [==============================] - 163s 1s/step - loss: 5.2284
Epoch 6/100
154/154 [==============================] - 166s 1s/step - loss: 5.0465
Epoch 7/100
154/154 [==============================] - 162s 1s/step - loss: 4.8729
Epoch 8/100
154/154 [==============================] - 178s 1s/step - loss: 4.7081
Epoch 9/100
154/154 [==============================] - 196s 1s/step - loss: 4.5479
Epoch 10/100
154/154 [==============================] - 201s 1s/step - loss: 4.3898
Epoch 11/100
154/154 [==============================] - 196s 1s/step - loss: 4.2320
Epoch 12/100
154/154 [==============================] - 198s 1s/step - loss: 4.0800
E

154/154 [==============================] - 203s 1s/step - loss: 0.0135
Epoch 99/100
154/154 [==============================] - 196s 1s/step - loss: 0.0112
Epoch 100/100
154/154 [==============================] - 207s 1s/step - loss: 0.0098


In [167]:
model.save_weights('nmt_weights.h5')

In [169]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [170]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [171]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [179]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input English sentence: by the summer in the middle of the general campaign
Actual Hindi Translation:  गर्मीयों तक आम चुनाव प्रचार के बीच में 
Predicted Hindi Translation:  गर्मीयों तक आम चुनाव प्रचार के बीच में 


In [180]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 25ms/step
Input English sentence: to conceal a secret within its seams or something
Actual Hindi Translation:  अपने सिलाई के भीतर एक राज़ छुपाने की या और कुछ 
Predicted Hindi Translation:  अपने सिलाई के भीतर एक राज़ छुपाने की या और कुछ 


In [181]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input English sentence: but i very much appreciate
Actual Hindi Translation:  पर मैं उसका बहुत आभारी हूँ कि 
Predicted Hindi Translation:  पर मैं उसका बहुत आभारी हूँ कि 


In [182]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 25ms/step
Input English sentence: to experience what it means to be a child laborer
Actual Hindi Translation:  ताकि वे अनुभव कर सकें कि बाल श्रमिक होना क्या होता है 
Predicted Hindi Translation:  ताकि वे अनुभव कर सकें कि बाल श्रमिक होना क्या 


In [183]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input English sentence: my friend chris who i just showed you a picture of
Actual Hindi Translation:  मेरा दोस्त क्रिस जिसके साथ खींची फोटो मैंने आपको अभी दिखाई थी 
Predicted Hindi Translation:  मेरा दोस्त क्रिस जिसके साथ खींची फोटो मैंने आपको
